In [ ]:
# Mount Drive to access your saved model
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Check your checkpoint exists
!ls /content/drive/MyDrive/wav2vec2-persian/

checkpoint-3504  checkpoint-5256  runs


In [ ]:
# Install libraries
!pip install transformers datasets evaluate torchaudio pyctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.1/538.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 127.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is

     - 553.6 kB 9.3 MB/s 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip


  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp312-cp312-linux_x86_64.whl size=3187999 sha256=105cb0044fdf784f82e1b474c209993902c6a5dded60adc731e7f5105dba3537
  Stored in directory: /tmp/pip-ephem-wheel-cache-4nvnnm1f/wheels/92/c8/12/56d187154e078f0eaa74d059017fc1afe1c4d91fbce02ce8d9
Successfully built kenlm


In [ ]:
# Load everything
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import torch
import torchaudio

In [ ]:
# Load model from your checkpoint
model_name = "m3hrdadfi/wav2vec2-large-xlsr-persian-v3"
checkpoint_path = "/content/drive/MyDrive/wav2vec2-persian/checkpoint-5256"

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(checkpoint_path)
processor = Wav2Vec2Processor.from_pretrained(model_name)
model.eval()
model = model.to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
# Load test dataset
ds_test_raw = load_dataset("hezarai/common-voice-13-fa", split="test")

In [ ]:
# Download KenLM
from huggingface_hub import hf_hub_download

lm_file = hf_hub_download(
    repo_id="mlengineer-ai/kenlm-sp-jomleh",
    filename="files/jomleh-sp-32000-o3-prune011.probing",  # 32K vocab, 3-gram
    repo_type="model"
)

In [ ]:
!pip install pyctcdecode

In [ ]:
# Build decoder
from pyctcdecode import build_ctcdecoder

vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab = sorted(vocab_dict.items(), key=lambda x: x[1])
vocab_list = [x[0] for x in sorted_vocab]

decoder = build_ctcdecoder(
    labels=vocab_list,
    kenlm_model_path=lm_file,
)

print("✅ Everything loaded! Ready for inference with LM")

✅ Everything loaded! Ready for inference with LM


In [ ]:
# Test inference with LM
def transcribe_with_lm(audio_array):
    inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt").to("cuda")

    with torch.no_grad():
        logits = model(**inputs).logits

    # Decode with LM
    beam_results = decoder.decode_beams(logits.cpu().numpy()[0])
    transcription = beam_results[0][0]  # Best hypothesis

    return transcription

# Test on some examples
print("🟢 Testing WITH Language Model:\n")

for i in range(10):
    audio = ds_test_raw[i]["audio"]["array"]
    sampling_rate = ds_test_raw[i]["audio"]["sampling_rate"]
    reference = ds_test_raw[i]["sentence"]

    if sampling_rate != 16000:
        audio = torchaudio.functional.resample(
            torch.tensor(audio), sampling_rate, 16000
        ).numpy()

    prediction = transcribe_with_lm(audio)

    print(f"Reference:  {reference}")
    print(f"Prediction: {prediction}")
    print()

🟢 Testing WITH Language Model:

Reference:  اتوبوس مسافری
Prediction: اتوبوس مسافری

Reference:  انعکاس مثبت دهید و اعتبار ببخشید
Prediction: انعکاس مثبت دهید و اعتبار ببخشید⁇

Reference:  جنگ افزارهای ساده
Prediction: جنگ ابزارهای سازه

Reference:  آیا حضرت عالی مرا صدا زدید؟
Prediction: آیا حضرت عالی مراصدازدید⁇

Reference:  باید باهاش حرف بزنم -
Prediction: با⁇د باهاش حرف بزنم

Reference:  غدد لنفاوی من ورم کرده اند.
Prediction: عدد لنفاویمن ویمکایدهاند⁇

Reference:  تو این دنیا یکی دلیل زندگیمه
Prediction: تو این دنیا⁇یکی دلیل زندگیمه

Reference:  من خیلی بهتون علاقه دارم و ازتون میخوام که اگه شما هم منو میپسندین و بهم علاقه دارین بیشتر اشنا بشیم
Prediction: من خیلی بهتون علاقه دارموازتون میخوام که اگه شما هم منو می پسندی و به م علاقه دارین بیشتر آشنا بشیم

Reference:  و خیلیها مادر شدند وقتی خودشان هنوز بچه بودند
Prediction: و خیلی همادهاسدقتیگفتان هنوز بت دند

Reference:  می خوام بچه دار بشم
Prediction: میخوام بچه دار بشم⁇



In [ ]:
import evaluate
from tqdm import tqdm

# Load metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

# Evaluate with LM on full test set
predictions_with_lm = []
references = []

print("Evaluating with LM on test set...")

for i in tqdm(range(len(ds_test_raw))):
    audio = ds_test_raw[i]["audio"]["array"]
    sampling_rate = ds_test_raw[i]["audio"]["sampling_rate"]
    reference = ds_test_raw[i]["sentence"]

    # Resample if needed
    if sampling_rate != 16000:
        audio = torchaudio.functional.resample(
            torch.tensor(audio), sampling_rate, 16000
        ).numpy()

    # Transcribe with LM
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt").to("cuda")

    with torch.no_grad():
        logits = model(**inputs).logits

    beam_results = decoder.decode_beams(logits.cpu().numpy()[0])
    prediction = beam_results[0][0]

    predictions_with_lm.append(prediction)
    references.append(reference)

# Calculate metrics
wer_with_lm = wer_metric.compute(predictions=predictions_with_lm, references=references)
cer_with_lm = cer_metric.compute(predictions=predictions_with_lm, references=references)

print("\n" + "="*50)
print("📊 RESULTS COMPARISON:")
print("="*50)
print(f"WITHOUT LM - WER: 0.2749 | CER: 0.1452")
print(f"WITH LM    - WER: {wer_with_lm:.4f} | CER: {cer_with_lm:.4f}")
print("="*50)

# Calculate improvement
wer_improvement = (0.2749 - wer_with_lm) / 0.2749 * 100
cer_improvement = (0.1452 - cer_with_lm) / 0.1452 * 100

print(f"\n✅ WER Improvement: {wer_improvement:.1f}%")
print(f"✅ CER Improvement: {cer_improvement:.1f}%")

Evaluating with LM on test set...


100%|██████████| 10440/10440 [16:50<00:00, 10.33it/s]



📊 RESULTS COMPARISON:
WITHOUT LM - WER: 0.2749 | CER: 0.1452
WITH LM    - WER: 0.4176 | CER: 0.1012

✅ WER Improvement: -51.9%
✅ CER Improvement: 30.3%


In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.0 MB/s eta 0:00:00


In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:00
